# Hyperparameter Tuning Demo using Keras Tuner

by Cedric Conol

Notebook from the article: [Hyperparameter Tuning with Keras Tuner](https://towardsdatascience.com/hyperparameter-tuning-with-keras-tuner-283474fbfbe)

### Load the data

In [111]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

### Check tensorflow and kerastuner versions

In [198]:
import tensorflow as tf
import kerastuner as kt

print(tf.__version__)
print(kt.__version__)

2.1.0
1.0.1


### Import libraries

In [186]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers
from kerastuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization

### Set random seeed

In [121]:
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

### Model without Hyperparameter Tuning

In [188]:
scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],),
                       kernel_initializer='zeros', bias_initializer='zeros'))
model.add(layers.Dense(16, activation='relu', kernel_initializer='zeros',
                       bias_initializer='zeros'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))

model.compile(optimizer='rmsprop',loss='mse',metrics=['mse'])

In [189]:
history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=10)

Train on 323 samples, validate on 81 samples
Epoch 1/10
323/323 [==============================] - ETA: 3s - loss: 578.7547 - mse: 578.754 - 0s 1ms/sample - loss: 571.3393 - mse: 571.3394 - val_loss: 642.7779 - val_mse: 642.7779
Epoch 2/10
323/323 [==============================] - ETA: 0s - loss: 500.8954 - mse: 500.895 - 0s 168us/sample - loss: 570.7098 - mse: 570.7098 - val_loss: 642.2042 - val_mse: 642.2042
Epoch 3/10
323/323 [==============================] - ETA: 0s - loss: 794.2568 - mse: 794.256 - 0s 164us/sample - loss: 570.1990 - mse: 570.1991 - val_loss: 641.6615 - val_mse: 641.6615
Epoch 4/10
323/323 [==============================] - ETA: 0s - loss: 530.8656 - mse: 530.865 - 0s 166us/sample - loss: 569.6986 - mse: 569.6985 - val_loss: 641.1320 - val_mse: 641.1320
Epoch 5/10
323/323 [==============================] - ETA: 0s - loss: 603.2223 - mse: 603.222 - 0s 162us/sample - loss: 569.2084 - mse: 569.2085 - val_loss: 640.6206 - val_mse: 640.6207
Epoch 6/10
323/323 [=======

In [199]:
mse = model.evaluate(x_test_scaled, y_test)[1]

102/102 [==============================] - ETA: 0s - loss: 499.3648 - mse: 499.364 - 0s 117us/sample - loss: 610.4096 - mse: 610.4095


In [204]:
print('MSE without tuning: {}'.format(mse))

MSE without tuning: 610.4095458984375


### Building the Hypermodel

In [206]:
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = Sequential()
        model.add(
            layers.Dense(
                units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape,
                kernel_initializer='zeros', bias_initializer='zeros'
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                kernel_initializer='zeros', bias_initializer='zeros'
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros'))
        
        model.compile(
            optimizer='rmsprop',loss='mse',metrics=['mse']
        )
        
        return model

### Initialize hypermodel

In [207]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

### Build Random Search Tuner

In [208]:
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials=10,
            executions_per_trial=2, overwrite=True
        )

### Run Random Search

In [209]:
tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

INFO:tensorflow:Oracle triggered exit


### Evaluate Random Search

In [210]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = best_model.evaluate(x_test_scaled, y_test)[1]

102/102 [==============================] - ETA: 0s - loss: 455.3785 - mse: 455.378 - 0s 727us/sample - loss: 561.8194 - mse: 561.8195


In [216]:
print('Random search MSE: ', mse_rs)

Random search MSE:  561.81946


### Build, Run and Evaluate Hyperband Tuner

In [218]:
tuner_hb = Hyperband(
            hypermodel,
            max_epochs=5,
            objective='mse',
            seed=42,
            executions_per_trial=2,
            directory='hb'
        )

tuner_hb.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model = tuner_hb.get_best_models(num_models=1)[0]
mse_hb = best_model.evaluate(x_test_scaled, y_test)[1]

INFO:tensorflow:Reloading Oracle from existing project hb/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from hb/untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit
102/102 [==============================] - ETA: 0s - loss: 316.5978 - mse: 316.597 - 0s 623us/sample - loss: 395.1894 - mse: 395.1895


In [219]:
print('Hyperband MSE: ', mse_hb)

Hyperband MSE:  395.18945


### Build, Run and Evaluate Bayesian Optimization Tuner

In [220]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2,
            directory='bo'
        )

tuner_bo.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model = tuner_bo.get_best_models(num_models=1)[0]
mse_bo = best_model.evaluate(x_test_scaled, y_test)[1]

INFO:tensorflow:Reloading Oracle from existing project bo/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from bo/untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit
102/102 [==============================] - ETA: 0s - loss: 38.3459 - mse: 38.345 - 0s 633us/sample - loss: 46.4670 - mse: 46.4670


In [221]:
print('Bayesian Optimization MSE: ', mse_bo)

Bayesian Optimization MSE:  46.466965
